# Parse an ILO data structure definition

### Load required libraries

In [1]:
import xmltodict
import json
import requests 
import xml.etree.ElementTree as ET 

import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import io

import math

### Interactive shell

In [2]:
# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1. Get data from API endpoint, save as xml file on disk, and parse using `xmltodict.parse`

In [3]:
def get_dsd(dataflow):

    baseURL = 'https://www.ilo.org/sdmx/rest/datastructure/ILO/'

    # url of API endpoint returning all the dissemination dataflows (i.e. indicators) with data available
    url = baseURL + dataflow + '?references=all' # 

    # creating HTTP response object from given url 
    resp = requests.get(url) 

    # saving the xml message into an xml file 
    with open('DSD__'+dataflow+'.xml', 'wb') as f: 
        f.write(resp.content) 
        
    with open('DSD__'+dataflow+'.xml', encoding="utf-8") as fd:
        doc = xmltodict.parse(fd.read())

    return doc

In [4]:

test_dsd = get_dsd('YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT')


#### Explore the contents of the first levels of the xml file

In [5]:
test_dsd.keys()

odict_keys(['message:Structure'])

In [6]:
test_dsd['message:Structure'].keys()

odict_keys(['@xmlns:message', '@xmlns:structure', '@xmlns:common', 'message:Header', 'message:Structures'])

In [7]:
test_dsd['message:Structure']['message:Structures'].keys()

odict_keys(['structure:OrganisationSchemes', 'structure:Dataflows', 'structure:Codelists', 'structure:Concepts', 'structure:DataStructures'])

### 2. Get the data pertaining to the dataflow

In [8]:
def dataflow_info(dsd):

    dataflow = dsd['message:Structure']['message:Structures']['structure:Dataflows']['structure:Dataflow']
    #dataflow
    dataflow_id = dataflow['@id']
    dataflow_agencyID = dataflow['@agencyID']
    for dfn in dataflow['common:Name']:
        if dfn['@xml:lang'] == 'en':
            dataflow_name = (dfn['#text'])
    for dfn in dataflow['common:Description']:
        if dfn['@xml:lang'] == 'en':
            dataflow_description = (dfn['#text'])

    return {'dataflow_id': dataflow_id,
            'dataflow_agencyID' : dataflow_agencyID,
            'dataflow_name': dataflow_name,
            'dataflow_description' : dataflow_description}

In [9]:
#dataflow_info(get_dsd('YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT'))

### 3. Get the code lists pertaining to the dataflow

In [10]:
def codelists(dsd):

    codelists = []

    for cl in dsd['message:Structure']['message:Structures']['structure:Codelists']['structure:Codelist']:

        codelist = {}
        codelist_id = cl['@id']

        # Get the code list name in English:
        for cln in cl['common:Name']:
            if cln['@xml:lang'] == 'en':
                codelist_name = (cln['#text'])

        codelist['id'] = codelist_id
        codelist['name'] = codelist_name
        codelist['codes'] = []

        # Some code lists have only one code; in this case, they are returned as a single dictionary. Other code
        # lists have many codes, and in this case they are returned as lists of dictionaries. 

        if isinstance(cl['structure:Code'], list):
            for c in cl['structure:Code']:
                for cn in c['common:Name']:
                    if cn['@xml:lang'] == 'en':
                        code_desc = (cn['#text'])

                        codelist['codes'].append({'code' : c['@id'], 'description' : code_desc})
        else:
            for cn in cl['structure:Code']['common:Name']:
                if cn['@xml:lang'] == 'en':
                    code_desc = (cn['#text'])

                    codelist['codes'].append({'code': cl['structure:Code']['@id'], 'description' : code_desc})

        codelists.append(codelist)

    # Display first four code lists
    return codelists


In [11]:
#codelists(get_dsd('YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT'))[0]

### 4. Get the concepts (and their respective code list ids) pertaining to the data flow

In [12]:
def concepts(dsd):

    concept_schemes = dsd['message:Structure']['message:Structures']['structure:Concepts']['structure:ConceptScheme']

    concepts = []

    for cs in concept_schemes:

        for c in cs['structure:Concept']:
            concept = {}

            concept['id'] = c['@id']

            # Get the concept scheme name in English:
            for cn in c['common:Name']:
                if cn['@xml:lang'] == 'en':
                    concept['name'] = (cn['#text'])
                    concept['scheme'] =  cs['@id']



            # Check if c contains any entry with key 'structure:CoreRepresentation' 
            if 'structure:CoreRepresentation' in c:
                concept['code_list'] = c['structure:CoreRepresentation']['structure:Enumeration']['Ref']['@id']
            else:
                concept['code_list'] = None

            concepts.append(concept)

    return concepts


In [13]:
#concepts(get_dsd('YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT'))

### 5. Get data structures

In [14]:
def structure(dsd):
        
    dimensionList =  dsd['message:Structure']['message:Structures']['structure:DataStructures']['structure:DataStructure']['structure:DataStructureComponents']['structure:DimensionList']
    attributeList =  dsd['message:Structure']['message:Structures']['structure:DataStructures']['structure:DataStructure']['structure:DataStructureComponents']['structure:AttributeList']
    measureList =  dsd['message:Structure']['message:Structures']['structure:DataStructures']['structure:DataStructure']['structure:DataStructureComponents']['structure:MeasureList']

    dimensions = []
    for d in dimensionList['structure:Dimension']:
        dimensions.append(d['@id'])                      
    #display(dimensions)


    attributes = []
    for a in attributeList['structure:Attribute']:
        attributes.append(a['@id'])                      
    #display(attributes)

    measures = []
    measures.append(measureList['structure:PrimaryMeasure']['@id'])
    #display(measures)

    structure = {'dimensions' : dimensions,
                 'attributes' : attributes,
                 'measures' : measures}

    return structure

In [15]:
#structure(get_dsd('YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT'))

### 6. Pack everything together

In [16]:
def parse_dsd(dsd):

    dsd2 = dataflow_info(dsd)

    dsd2['concepts'] = concepts(dsd)
    dsd2['structure'] = structure(dsd)
    dsd2['codelists'] = codelists(dsd)

    return(dsd2)



In [17]:
#parse_dsd(get_dsd('YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT'))

## Read data in csv format

In [18]:
def parse_data(dataflow,start,end):
    
    baseURL = 'https://www.ilo.org/sdmx/rest/data/ILO,DF_'
    url = baseURL + dataflow + '/?format=csv&startPeriod=' + start + '&endPeriod=' + end# 
    s=requests.get(url).content
    
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    return c

In [19]:
start = '2000-01-01'
end = '2019-12-31'
dataflow = 'YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT'

x = parse_data(dataflow, start, end)
x.head(5)
x.columns

,DATAFLOW,COLLECTION,REF_AREA,FREQ,SURVEY,MEASURE,SEX,AGE,GEO,TIME_PERIOD,...,T2,I8,OBS_STATUS,UNIT_MEASURE_TYPE,UNIT_MEASURE,UNIT_MULT,FREE_TEXT_NOTE,DECIMALS,SOURCE,INDICATOR
0,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2002,...,NaN,NaN,NaN,RT,PT,0,NaN,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
1,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2003,...,NaN,NaN,NaN,RT,PT,0,NaN,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
2,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2004,...,NaN,NaN,NaN,RT,PT,0,NaN,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
3,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2005,...,NaN,NaN,NaN,RT,PT,0,NaN,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
4,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2006,...,NaN,NaN,NaN,RT,PT,0,NaN,1,BA,EAP_DWAP_SEX_AGE_GEO_RT


Index(['DATAFLOW', 'COLLECTION', 'REF_AREA', 'FREQ', 'SURVEY', 'MEASURE',
       'SEX', 'AGE', 'GEO', 'TIME_PERIOD', 'OBS_VALUE', 'S3', 'T3', 'C12',
       'I13', 'S4', 'R1', 'T5', 'C6', 'S14', 'I11', 'S5', 'T2', 'I8',
       'OBS_STATUS', 'UNIT_MEASURE_TYPE', 'UNIT_MEASURE', 'UNIT_MULT',
       'FREE_TEXT_NOTE', 'DECIMALS', 'SOURCE', 'INDICATOR'],
      dtype='object')

## Verify existence of duplicates due to multiple surveys

In [20]:
test2 = x[['DATAFLOW', 'COLLECTION', 'REF_AREA', 'FREQ', 'MEASURE','SEX', 'AGE', 'GEO', 'TIME_PERIOD']]
test2.shape

test1 = x[['DATAFLOW', 'COLLECTION', 'REF_AREA', 'FREQ', 'MEASURE','SEX', 'AGE', 'GEO', 'TIME_PERIOD']].drop_duplicates()
test1.shape

if test1.shape != test2.shape:
    print("There are instances of multiple surveys in the same year")

(93608, 9)

(91355, 9)

There are instances of multiple surveys in the same year


### Pick only the survey with the maximum survey ID

In [21]:
idx = x.groupby(['DATAFLOW', 'COLLECTION', 'REF_AREA', 'FREQ', 'MEASURE', 'SEX', 'AGE', 'GEO', 'TIME_PERIOD'])['SURVEY'].transform(max) == x['SURVEY']
x = x[idx]
x['FREE_TEXT_NOTE'] = x['FREE_TEXT_NOTE'].astype(str)
x.head(5)

,DATAFLOW,COLLECTION,REF_AREA,FREQ,SURVEY,MEASURE,SEX,AGE,GEO,TIME_PERIOD,...,T2,I8,OBS_STATUS,UNIT_MEASURE_TYPE,UNIT_MEASURE,UNIT_MULT,FREE_TEXT_NOTE,DECIMALS,SOURCE,INDICATOR
0,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2002,...,NaN,NaN,NaN,RT,PT,0,nan,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
1,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2003,...,NaN,NaN,NaN,RT,PT,0,nan,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
2,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2004,...,NaN,NaN,NaN,RT,PT,0,nan,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
3,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2005,...,NaN,NaN,NaN,RT,PT,0,nan,1,BA,EAP_DWAP_SEX_AGE_GEO_RT
4,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2006,...,NaN,NaN,NaN,RT,PT,0,nan,1,BA,EAP_DWAP_SEX_AGE_GEO_RT


In [22]:
x.columns

Index(['DATAFLOW', 'COLLECTION', 'REF_AREA', 'FREQ', 'SURVEY', 'MEASURE',
       'SEX', 'AGE', 'GEO', 'TIME_PERIOD', 'OBS_VALUE', 'S3', 'T3', 'C12',
       'I13', 'S4', 'R1', 'T5', 'C6', 'S14', 'I11', 'S5', 'T2', 'I8',
       'OBS_STATUS', 'UNIT_MEASURE_TYPE', 'UNIT_MEASURE', 'UNIT_MULT',
       'FREE_TEXT_NOTE', 'DECIMALS', 'SOURCE', 'INDICATOR'],
      dtype='object')

### Parse DSD for the dataflow

In [23]:
dsd = parse_dsd(get_dsd(dataflow))
dsd.keys()

dict_keys(['dataflow_id', 'dataflow_agencyID', 'dataflow_name', 'dataflow_description', 'concepts', 'structure', 'codelists'])

### Add code description to each coded dimension 

In [24]:
for d in dsd['structure']['dimensions']:
    for c in dsd['concepts']:
        if d != c['id']:
            continue
        codelist_id = c['code_list']
        
        #print('-----------------')
        #print(d)
        #print(codelist_id)
        #print('-----------------')
        
        for cl in dsd['codelists']:
            if codelist_id != cl['id']:
                continue
                
            # if dimension is numerit, to str
            
            cl_dict = pd.DataFrame.from_dict(cl['codes'])
            cl_dict.columns = [d, d+'_DESC']
            
            if is_numeric_dtype(x[d]):
                cl_dict[d] = cl_dict[d].astype(float)
            
            #cl_dict.head(5)
            
            x = pd.merge(x,
                 cl_dict,
                 on=d, 
                 how='left')

        
        


### Add code description to each coded attribute

In [25]:
for a in dsd['structure']['attributes']:
    for c in dsd['concepts']:
        if a != c['id']:
            continue
        codelist_id =c['code_list']
        
        #print('-----------------')
        #print(d)
        #print(codelist_id)
        #print('-----------------')
        
        for cl in dsd['codelists']:
            if codelist_id != cl['id']:
                continue
                

            
            cl_dict = pd.DataFrame.from_dict(cl['codes'])
            cl_dict.columns = [a, a+'_DESC']
            
            # if attribute is numerit 
            if is_numeric_dtype(x[a]):
                cl_dict[a] = cl_dict[a].astype(float)
            
            #cl_dict.head(5)
            
            x = pd.merge(x,
                 cl_dict,
                 on=a, 
                 how='left')


In [26]:
x.columns
x.head(5)


Index(['DATAFLOW', 'COLLECTION', 'REF_AREA', 'FREQ', 'SURVEY', 'MEASURE',
       'SEX', 'AGE', 'GEO', 'TIME_PERIOD', 'OBS_VALUE', 'S3', 'T3', 'C12',
       'I13', 'S4', 'R1', 'T5', 'C6', 'S14', 'I11', 'S5', 'T2', 'I8',
       'OBS_STATUS', 'UNIT_MEASURE_TYPE', 'UNIT_MEASURE', 'UNIT_MULT',
       'FREE_TEXT_NOTE', 'DECIMALS', 'SOURCE', 'INDICATOR', 'COLLECTION_DESC',
       'REF_AREA_DESC', 'FREQ_DESC', 'SURVEY_DESC', 'MEASURE_DESC', 'SEX_DESC',
       'AGE_DESC', 'GEO_DESC', 'S3_DESC', 'T3_DESC', 'C12_DESC', 'I13_DESC',
       'S4_DESC', 'R1_DESC', 'T5_DESC', 'C6_DESC', 'S14_DESC', 'I11_DESC',
       'S5_DESC', 'T2_DESC', 'I8_DESC', 'OBS_STATUS_DESC',
       'UNIT_MEASURE_TYPE_DESC', 'UNIT_MEASURE_DESC', 'UNIT_MULT_DESC',
       'SOURCE_DESC', 'INDICATOR_DESC'],
      dtype='object')

,DATAFLOW,COLLECTION,REF_AREA,FREQ,SURVEY,MEASURE,SEX,AGE,GEO,TIME_PERIOD,...,I11_DESC,S5_DESC,T2_DESC,I8_DESC,OBS_STATUS_DESC,UNIT_MEASURE_TYPE_DESC,UNIT_MEASURE_DESC,UNIT_MULT_DESC,SOURCE_DESC,INDICATOR_DESC
0,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2002,...,NaN,NaN,NaN,NaN,NaN,Rate,Percentage,Units,Labour force survey,"Labour force participation rate by sex, age an..."
1,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2003,...,NaN,NaN,NaN,NaN,NaN,Rate,Percentage,Units,Labour force survey,"Labour force participation rate by sex, age an..."
2,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2004,...,NaN,NaN,NaN,NaN,NaN,Rate,Percentage,Units,Labour force survey,"Labour force participation rate by sex, age an..."
3,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2005,...,NaN,NaN,NaN,NaN,NaN,Rate,Percentage,Units,Labour force survey,"Labour force participation rate by sex, age an..."
4,ILO:DF_YI_ALL_EAP_DWAP_SEX_AGE_GEO_RT(1.0),YI,PRT,A,2252,EAP_DWAP_RT,SEX_T,AGE_AGGREGATE_TOTAL,GEO_COV_NAT,2006,...,NaN,NaN,NaN,NaN,NaN,Rate,Percentage,Units,Labour force survey,"Labour force participation rate by sex, age an..."


### Concatenate coded notes and free text footnote

In [27]:
for index, row in x.iterrows():
    footnote = ''
    for c in dsd['concepts']:
        if c['scheme'] != 'CS_NOTE_TYPE':
            continue
        # access data using column names
        if not math.isnan(row[c['id']]):
            footnote = footnote + c['name']+': '+row[c['id']+'_DESC'] + '. '
            
    if x.at[index, 'FREE_TEXT_NOTE'] == 'nan':
        x.at[index, 'FREE_TEXT_NOTE'] = footnote
    else:
         x.at[index, 'FREE_TEXT_NOTE'] = footnote + x.at[index, 'FREE_TEXT_NOTE']


### Select columns to save into excel

In [35]:
#x.columns
#dsd['structure']['dimensions']
#dsd['structure']['attributes']
#dsd['structure']['measures']

final_columns = ['DATAFLOW']

# Add dimension columns:
for d in dsd['structure']['dimensions']:
    final_columns.append(d)
    final_columns.append(d+'_DESC')

# Add time dimension column:
final_columns.append('TIME_PERIOD')

# Add primary measure column:
final_columns = final_columns + dsd['structure']['measures']

# Add attribute columns (except coded notes):
for a in dsd['structure']['attributes']:
    for c in dsd['concepts']:
        if c['scheme'] == 'CS_NOTE_TYPE':
            continue
        if a == c['id']:
            final_columns.append(a)
            if a+'_DESC' in x.columns:
                final_columns.append(a+'_DESC')

x = x.sort_values(by=['REF_AREA', 'SEX', 'AGE', 'GEO', 'TIME_PERIOD'])
x[final_columns].to_excel('test.xlsx')     

In [34]:
dsd['structure']['dimensions']

['COLLECTION', 'REF_AREA', 'FREQ', 'SURVEY', 'MEASURE', 'SEX', 'AGE', 'GEO']